In [1]:
import joblib
import sys
import os
alamat = "/".join(os.getcwd().split("\\")[:-2])
sys.path.append(alamat+'/praproses')
import ngrams as ng
from normalisasi import normalisasi as nrm
import token as tkn
import stemming as stm
import cleansing
import stemming
import seleksi_kata as sk
import pandas as pd
import isDigit as isd

In [2]:
tdidf = joblib.load("model/tdidf.w")
naive_bayes = joblib.load("model/naive_bayes.m")

In [3]:
fitur = tdidf.get_feature_names()
kelas  = naive_bayes.classes_
kelas
postif = naive_bayes.feature_log_prob_ [1]
negatif = naive_bayes.feature_log_prob_ [0]
dict_ = {
    'features':fitur,
    'positive':postif,
    'negative':negatif,
}
df_bobot = pd.DataFrame.from_dict(dict_)
df_bobot.to_excel("hasil_training.xlsx")
df_bobot

,features,positive,negative
0,01,-7.733685,-9.460691
1,01caprespendustaabadi,-8.128291,-7.132576
2,01dila,-8.232101,-9.460691
3,01dilanj,-8.018842,-9.460691
4,01dilanjutkan,-5.897480,-7.291395
...,...,...,...
2371,zalim,-9.442547,-7.846073
2372,zaman,-8.226094,-9.460691
2373,zhandou,-8.291441,-9.460691
2374,znzz,-8.028081,-9.460691


In [4]:
def praproses(twt, n=1):
    twt = sk.seleksi(twt)
    twt = cleansing.cleaning(twt)
    twt = isd.remove_digit(twt)
    twt = nrm.slangword(twt)
    twt = stm.stemming(twt)
    twt = stm.StopWordRemover(twt)
    twt = ng.ngramku(twt, n=n)['string']
    return twt

In [5]:
praproses("RT @hpsetyawan: @afiadi setuju bgt", n=1)

'tuju banget'

In [6]:
kalimat = "RT @hpsetyawan: setuju bgt"
# print(praproses(kalimat))
naive_bayes.predict(tdidf.transform([praproses(kalimat)]))[0]
#memprediksi kalimat di preproses, kemudian di transform menjadi vector,diprediksi menggunakan NB

'positive'

In [7]:
# pwd

# Prediksi data uji [Merdeka Belajar]

In [8]:
df = pd.read_excel("../../data_uji/PPDB ZONASI.xlsx")
twet_uji = df.Twit.tolist()
label_asli = df.label.tolist()
twet_uji_bersih = list()

for ix, t in enumerate(twet_uji):
    twet_uji_bersih.append(praproses(t))
#     print(ix)
prediksi = naive_bayes.predict(tdidf.transform(twet_uji_bersih))

from sklearn.metrics import classification_report
y_true = label_asli
y_pred = prediksi
target_names = ['negative', 'positive',]
print(classification_report(y_true, y_pred, target_names=target_names))

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred,labels=["positive", "negative"])

In [ ]:
ix = 0
for twet, asli, pred in zip(twet_uji, label_asli,prediksi):
    if pred!= asli:
        print(ix)
        print("Prediksi:",pred)
        print("asli    :",asli)
        print("    ",twet)
        print("="*100)
    ix+=1

In [ ]:
import numpy as np

In [ ]:
np.log(0.02)